In [1]:
!pip install python-telegram-bot --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.9/654.9 kB 34.7 MB/s eta 0:00:00


In [2]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 kB 30.1 MB/s eta 0:00:00


In [12]:
import torch
from ultralytics import YOLO
from telegram.ext import Application, CommandHandler, MessageHandler, filters
from telegram import Update
from telegram.ext import ContextTypes
import os
import nest_asyncio
from io import BytesIO
import time

# إصلاح مشكلة حلقة asyncio
nest_asyncio.apply()

# التحقق من GPU
print("Number of GPU: ", torch.cuda.device_count())
if torch.cuda.is_available():
    print("GPU Name: ", torch.cuda.get_device_name())

# إعداد الجهاز
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# تحميل النموذج
MODEL_PATH = "/content/best.pt"  # تأكد من أن النموذج محفوظ هنا
model = YOLO(MODEL_PATH)

async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """الرد على أمر /start."""
    await update.message.reply_text(
        "مرحبًا! أنا BTumor بوت لتحليل الصور باستخدام YOLO. "
        "أرسل صورة ليتم تحليلها، وسأرسل لك الصورة المعالجة. 😊"
    )

async def image_handler(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """معالجة الصورة وإرسال الصورة المعالجة."""
    # رسالة تأكيد استلام الصورة
    await update.message.reply_text("تم استلام الصورة! 🚀 يتم الآن تجهيزها للتحليل...")

    # الحصول على الصورة المرسلة
    photo = update.message.photo[-1]
    photo_file = await photo.get_file()

    # تحميل الصورة كـ byte array
    input_image_path = "input.jpg"
    file_bytes = await photo_file.download_as_bytearray()

    # حفظ الصورة مؤقتًا
    with open(input_image_path, 'wb') as f:
        f.write(file_bytes)
    print(f"تم حفظ الصورة في: {input_image_path}")

    # حذف الصور القديمة في مجلد "runs/detect/predict/"
    output_dir = "runs/detect/"
    if os.path.exists(output_dir):
        for f in os.listdir(output_dir):
            file_path = os.path.join(output_dir, f)
            if os.path.isdir(file_path):  # حذف المجلدات الفرعية أيضًا
                for file in os.listdir(file_path):
                    os.remove(os.path.join(file_path, file))

    # رسالة إعلام بأن الصورة قيد المعالجة
    await update.message.reply_text("✋ يتم الآن تشغيل YOLO لتحليل الصورة. يرجى الانتظار قليلًا...")

    # تشغيل YOLO لتحليل الصورة
    try:
        results = model.predict(
            source=input_image_path,
            imgsz=416,
            conf=0.5,
            save=True  # حفظ النتيجة في مجلد "runs/detect/"
        )
        print("تم تحليل الصورة بنجاح.")
    except Exception as e:
        print(f"خطأ أثناء تحليل الصورة: {e}")
        await update.message.reply_text("❌ حدث خطأ أثناء تحليل الصورة. يرجى المحاولة مرة أخرى.")
        return

    # تأخير بسيط للتأكد من حفظ الصورة قبل التحقق منها
    time.sleep(3)

    # التحقق من المجلدات التي تحتوي على النتائج المحفوظة
    output_images = []
    result_dir = "runs/detect/"
    for subdir in os.listdir(result_dir):
        subdir_path = os.path.join(result_dir, subdir)
        if os.path.isdir(subdir_path):
            output_images = os.listdir(subdir_path)
            if output_images:
                print(f"تم العثور على المجلد الفرعي: {subdir}")
                break

    if not output_images:
        await update.message.reply_text("❌ لم يتم العثور على صورة محللة. يرجى المحاولة مرة أخرى.")
        return

    # تحديد مسار الصورة المحللة
    output_image_path = os.path.join(result_dir, subdir, output_images[0])

    # التحقق من أن الصورة المحللة ليست فارغة
    if os.path.getsize(output_image_path) == 0:
        print("الصورة المحللة فارغة!")
        await update.message.reply_text("❌ حدث خطأ أثناء معالجة الصورة. يرجى المحاولة مرة أخرى.")
        return

    print(f"تم العثور على الصورة المحللة في: {output_image_path}")

    # إرسال الصورة المحللة
    with open(output_image_path, "rb") as result_image:
        image_bytes = BytesIO(result_image.read())
        await update.message.reply_photo(photo=image_bytes, caption="🎉 هذه هي الصورة المحللة! 😊")

    # رسالة وداع مع اقتراح
    await update.message.reply_text(
        "👍 إذا أردت تحليل صورة أخرى، لا تتردد في إرسالها الآن. شكرًا لاستخدامك BTumor! 🌟"
    )

# إعداد البوت
BOT_TOKEN = "7618729376:AAF3F1NEaceb84-bSyd34QV21zJsLYsKd5E"  # تأكد من تحديث التوكن
app = Application.builder().token(BOT_TOKEN).build()

# ربط الأوامر
app.add_handler(CommandHandler("start", start))

# ربط معالجة الصور
app.add_handler(MessageHandler(filters.PHOTO, image_handler))

# بدء تشغيل البوت
print("Running bot...")
app.run_polling()


Number of GPU:  1
GPU Name:  Tesla T4
Using device: cuda
Running bot...
تم حفظ الصورة في: input.jpg

image 1/1 /content/U-2-Net/input.jpg: 416x416 1 meningioma, 10.7ms
Speed: 1.7ms preprocess, 10.7ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 416)
Results saved to runs/detect/predict4
تم تحليل الصورة بنجاح.
تم العثور على المجلد الفرعي: predict4
تم العثور على الصورة المحللة في: runs/detect/predict4/input.jpg


RuntimeError: Cannot close a running event loop